<a href="https://colab.research.google.com/github/AbdullahNasser98/Tahaluf-project/blob/master/Task-1%20(YoloX)/yoloX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install yolox repo
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git
!cd YOLOX

Cloning into 'YOLOX'...
remote: Enumerating objects: 1806, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 1806 (delta 6), reused 11 (delta 3), pack-reused 1782
Receiving objects: 100% (1806/1806), 6.86 MiB | 22.96 MiB/s, done.
Resolving deltas: 100% (1071/1071), done.


In [2]:
%cd YOLOX

/content/YOLOX


In [3]:
# Install all dependencies
!pip install -U pip && pip install -r requirements.txt
!pip install -v -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 6.8 MB/s eta 0:00:00
Using pip 22.3.1 fr

Download Coco 128 dataset.

COCO 128 is a small dataset with 71 classes.

It is perfect for our use here since it is a small dataset, and thus won't take a long time to train.

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/gdrive/MyDrive/yolox_weights/COCO 128.v2-640x640.coco.zip', 'r')
zip_ref.extractall('/content/YOLOX/datasets')
zip_ref.close()

We are performing transfer learning on the yolox_s on our COCO128 dataset.

The model is expected to yield good results, since the dataset is similar to the dataset the model was trained on. Even higher results can be achieved if we use a bigger model, such as yolox_l, however the training time will increase significanlty, since the model parameters will increase.

In [13]:
!python tools/train.py -f exps/example/custom/yolox_s.py -d 1 -b 8 --fp16 -o -c /content/gdrive/MyDrive/yolox_weights/yolox_s.pth

2023-01-11 16:40:18 | INFO     | yolox.core.trainer:131 - args: Namespace(batch_size=8, cache=None, ckpt='/content/gdrive/MyDrive/yolox_weights/yolox_s.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='exps/example/custom/yolox_s.py', experiment_name='yolox_s', fp16=True, logger='tensorboard', machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2023-01-11 16:40:18 | INFO     | yolox.core.trainer:132 - exp value:
╒═══════════════════╤══════════════════════════════════════════════════════════╕
│ keys              │ values                                                   │
╞═══════════════════╪══════════════════════════════════════════════════════════╡
│ seed              │ None                                                     │
├───────────────────┼──────────────────────────────────────────────────────────┤
│ output_dir        │ './YOLOX_outputs'                                        │
├───────────────────┼───────────────────

As seen from the previous cell, after training the model for 50 epochs, the AP is **0.677**



Now we use the model weights from the above training to test the model

In [14]:
MODEL_PATH = "/content/YOLOX/YOLOX_outputs/yolox_s/latest_ckpt.pth"
!python3 tools/eval.py -n  yolox-s -c {MODEL_PATH} -b 8 -d 1 --conf 0.001 -f exps/example/custom/yolox_s.py

2023-01-11 16:53:01 | INFO     | __main__:139 - Args: Namespace(batch_size=8, ckpt='/content/YOLOX/YOLOX_outputs/yolox_s/latest_ckpt.pth', conf=0.001, devices=1, dist_backend='nccl', dist_url=None, exp_file='exps/example/custom/yolox_s.py', experiment_name='yolox_s', fp16=False, fuse=False, legacy=False, machine_rank=0, name='yolox-s', nms=None, num_machines=1, opts=[], seed=None, speed=False, test=False, trt=False, tsize=None)
2023-01-11 16:53:01 | INFO     | __main__:149 - Model Summary: Params: 8.97M, Gflops: 26.91
2023-01-11 16:53:01 | INFO     | __main__:150 - Model Structure:
YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
       

And yield the above percision and recall per class

Now we will move to Yolov5 and try training the model on similar conditions, and compare the results